In [2]:
import os
import random
import yaml

In [3]:
def mkdir(path):
    '''
    创建指定的文件夹
    :param path: 文件夹路径，字符串格式
    :return: True(新建成功) or False(文件夹已存在，新建失败)
    '''
    # 引入模块
    import os

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")

    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)

    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功\n')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在\n')
        return False

In [4]:
nanov7_dir = r'/data/pubfs/pku_visitor/public_write/download/datasets/nanov7'

isdata = 'mc'#'data' or 'mc'
if isdata=='data':
    Data_merge = ['DoubleMuon_Run2018A','DoubleMuon_Run2018B','DoubleMuon_Run2018C','DoubleMuon_Run2018D',
                 'EGamma_Run2018A','EGamma_Run2018B','EGamma_Run2018C','EGamma_Run2018D',
                 'SingleMuon_Run2018A','SingleMuon_Run2018B','SingleMuon_Run2018C','SingleMuon_Run2018D',
                 'MuonEG_Run2018A','MuonEG_Run2018B','MuonEG_Run2018C','MuonEG_Run2018D'
                 ]
    PROCESSES= [Data_merge]
    
    with open(f"/data/pubfs/tyyang99/jupyter_files/data_2018_nanov7.yaml", 'r') as f:
        mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
elif isdata=='mc':
    WW_polar = ['WpWpJJ_LL_polarization_EWK','WpWpJJ_TL_polarization_EWK','WpWpJJ_TT_polarization_EWK']
#     WW_merge = ['WpWpJJ_LL_polarization_EWK','WpWpJJ_EWK', 'WpWpJJ_QCD']
#     WZ_EWK = ['WLLJJ_EWK']
#     WZ_merge = ['WZTo3LNu_0Jets_MLL_4to50','WZTo3LNu_1Jets_MLL_4to50','WZTo3LNu_2Jets_MLL_4to50','WZTo3LNu_3Jets_MLL_4to50',
#                'WZTo3LNu_0Jets_MLL_50','WZTo3LNu_1Jets_MLL_50','WZTo3LNu_2Jets_MLL_50','WZTo3LNu_3Jets_MLL_50']
#     ZZ_merge = ['ZZJJTo4L_EWK','ZZJJTo4L_QCD','ZZJJTo4L_int','ggZZ_2e2mu',
#                'ggZZ_2e2tau','ggZZ_2mu2tau','ggZZ_4e','ggZZ_4mu','ggZZ_4tau','ggZZ_4tau_ext']
#     TVX_merge = ['TTGJets','TTZToQQ','TTZToQQ_ext','TTZToLLNuNu_M_10',
#                  'TTWJetsToQQ','TTWJetsToLNu','tZq']
#     VG_merge = ['Zgamma_EWK','Zgamma_EWK_ext','Wgamma_EWK','Wgamma_int']
#     WS_merge = ['WWTo2L2Nu',
#                 'ggWW_ee','ggWW_em','ggWW_et','ggWW_me','ggWW_mm','ggWW_mt','ggWW_te','ggWW_tm','ggWW_tt',
#                 'TTTo2L2Nu','ST_tW_top','ST_tW_antitop',
#                 'DYJetsToLL_M50','DYJets_M10to50','DYJets_M10to50_ext','ggh_ww','ggh_zz','ggh_tautau',
#                 'VBF_HToZZTo4L','VBFHToWWTo2L2Nu','VBFHToTauTau',
#                 'ttHToNonbb','VHToNonbb']
#     Other_merge = ['WW_DS','WWW','WWZ','WZZ','ZZZ','WWG']
    
#     PROCESSES = [WW_merge, WZ_EWK, WZ_merge, ZZ_merge, TVX_merge, VG_merge, WS_merge, Other_merge]
    PROCESSES = [WW_polar]
    
    with open(f"/data/pubfs/tyyang99/jupyter_files/changexs_datasets.yaml", 'r') as f:
        mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
else:
    print('!!!Wrong data type!!!')

In [5]:
output_dir = f'coffea_newid_{isdata}_ssww_parquet'
py_file = 'ssww.py'
pwd = os.popen('pwd').readlines()[0].replace('\n','')

for i,process_list in enumerate(PROCESSES):
    for process in process_list:
        print(process)
        mkdir(f'{output_dir}/{process}')
        process_dir = nanov7_dir + mc_yaml[process]['dataset']
        root_list = os.listdir(process_dir)
        for n_root,root_file in enumerate(root_list):
            root_dir = process_dir+'/'+root_file
            print(root_file,'%d/%d'%(n_root+1,len(root_list)))
            mkdir(f'{output_dir}/{process}/{str(n_root)}')
            command = 'python %s -n $2 -y 2018 -f $1 -t %s -o %s/%s/%d'%(pwd+'/'+py_file, isdata, pwd+'/'+output_dir, process, n_root)
            
            file_command = open('%s/%s/%d/command_%d.sh'%(output_dir, process, n_root, n_root),'w')
            file_command.write('#! /bin/bash\n\n')
            file_command.write('sleep %d\n'%(random.randint(300,1800)))
            file_command.write('source /home/pku/tyyang99/.bashrc\n')
            file_command.write('cd %s\n'%(pwd))
            file_command.write(command+'\n')
            file_command.write('echo "job done!"')
            file_command.close()
            
            file_submit = open('%s/%s/%d/submit_%d.txt'%(output_dir, process, n_root,n_root),'w')
            file_submit.write('Universe = vanilla\n')
            file_submit.write('Executable = %s/%s/%d/command_%d.sh\n'%(output_dir, process, n_root,n_root))
            file_submit.write('requirements = (machine == "node05.phy.pku.edu.cn") || (machine == "node06.phy.pku.edu.cn")\n')
            file_submit.write('arguments = %s %d\n'%(root_dir,n_root))
            file_submit.write('should_transfer_files = YES\n')
            file_submit.write('Error = %s/%s/%d/%d.err\n'%(output_dir, process, n_root,n_root))
            file_submit.write('Output = %s/%s/%d/%d.out\n'%(output_dir, process, n_root,n_root))
            file_submit.write('Log = %s/%s/%d/%d.log\n'%(output_dir, process, n_root,n_root))
            file_submit.write('Queue 1 \n')
            file_submit.close()
            
            condor_output = os.popen('condor_submit %s/%s/%d/submit_%d.txt'%(output_dir, process, n_root,n_root)).readlines()
            for line in condor_output:
                print(line,end='')
            print()

WpWpJJ_LL_polarization_EWK
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK 目录已存在

676FE04D-5148-734E-9641-B5C25DF4DA26.root 1/16
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK/0 目录已存在

Submitting job(s).
1 job(s) submitted to cluster 146813.

D2551835-7955-9944-9362-5648CB26A88E.root 2/16
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK/1 目录已存在

Submitting job(s).
1 job(s) submitted to cluster 146814.

D623C69E-8480-524E-975C-3E28085D7277.root 3/16
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK/2 目录已存在

Submitting job(s).
1 job(s) submitted to cluster 146815.

2DF4F118-4E33-C74C-A0A1-8015B4409DEE.root 4/16
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK/3 目录已存在

Submitting job(s).
1 job(s) submitted to cluster 146816.

E4C878F9-B201-8441-A0FA-582568579B01.root 5/16
coffea_newid_mc_ssww_parquet/WpWpJJ_LL_polarization_EWK/4 目录已存在

Submitting job(s).
1 job(s) submitted to cluster 146817.

4D095C2C-18DC-BB4F-AA88-7299EF5DA8FB.root 6/16
coffea_newid_